## Imports

Imports every library needed

In [1]:
#general imports
import pandas as pd
import numpy as np

#statsmodels for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

#scipy for testing
from scipy import stats

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

## Importing the Bike Data

Imports the normal Data needed for every KPI

In [2]:
df_bikes = pd.read_csv('../data/philadelphia_2017.csv')

df_bikes["start_time"] = pd.to_datetime(df_bikes["start_time"], dayfirst = True)

df_bikes["end_time"] = pd.to_datetime(df_bikes["end_time"], dayfirst = True)

df_bikes['ride_duration_minutes'] = df_bikes['end_time'] - df_bikes['start_time']

ride_lengths = []

for label, content in df_bikes.iterrows():
    ride_lengths.append(content["ride_duration_minutes"])

ride_lengths_np = np.array(ride_lengths)
iqr = stats.iqr(ride_lengths_np)

q1,q3 = np.percentile(ride_lengths_np, [25,99.9])

upper_range = q3 + (1.5*iqr)

df_bikes.drop(df_bikes[ (df_bikes.ride_duration_minutes > upper_range) | (df_bikes.end_time > pd.to_datetime("2017-12-31 23:59:59")) | (df_bikes.start_time > pd.to_datetime("2017-12-31 23:59:59")) | (df_bikes.ride_duration_minutes < pd.to_timedelta(0))].index , inplace=True)

In [3]:
df_weather = pd.read_csv('../data/weather_hourly_philadelphia.csv')

Change type of "date_time" to datetime

In [4]:
df_weather["date_time"] = pd.to_datetime(df_weather["date_time"])

Ordered weather data by date

In [5]:
df_weather = df_weather.sort_values(by=['date_time'])

Deleted weather date which is not needed for Philadelphia 2017 (<2017 or >2017)

In [6]:
start2017 = datetime(2016, 12, 31, 23)
end2017 = datetime(2018, 1, 1)

df_2017weather = df_weather[(df_weather["date_time"] > start2017) & (df_weather["date_time"] < end2017)]
df_2017weather_unique = df_2017weather.drop_duplicates(subset='date_time')
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

<ipython-input-6-92eb66279140>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))


Identify missing data and generate by using average data from former and following hour

In [7]:
for i in df_2017weather_unique.index:
    if i == 8231:
        pass
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

New order by date, because we added new lines

In [8]:
df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

Running the same code again because one line is missing

In [9]:
for i in df_2017weather_unique.index:
    if i == 8758:
        pass
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

New order by date, because we added new lines

In [30]:
df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')
df_2017weather_unique.head()

,date_time,max_temp,min_temp,precip
Order,,,,
0,2017-01-01 00:00:00,6.1,6.1,0.0
1,2017-01-01 01:00:00,6.1,6.1,0.0
2,2017-01-01 02:00:00,6.7,6.7,0.0
3,2017-01-01 03:00:00,7.2,7.2,0.0
4,2017-01-01 04:00:00,7.8,7.8,0.0


Check if data is complete by checking time difference of following

In [11]:
for i in df_2017weather_unique.index:
    if i == 8759:
        pass
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

def would_be_rounded_up(num):
    if round(num) == int(num) + 1:
        return True
    else:
        return False

def get_precip(start_time, end_time):
    return df_2017weather_unique["precip"][get_weather_df_row(start_time, end_time)]

def get_temperature(start_time, end_time):
    row = get_weather_df_row(start_time, end_time)
    return (df_2017weather_unique["min_temp"][row] + df_2017weather_unique["max_temp"][row])/2

def get_weather_df_row(start_time, end_time):
    duration_of_travel = end_time-start_time
    average_timestamp = end_time - datetime.fromisoformat("2017-01-01 00:00:00") - duration_of_travel/2
    weather_row = average_timestamp.total_seconds()/3600
    return int(weather_row)

df_bikes["temperature"] = df_bikes.apply(lambda x: get_temperature(x["start_time"], x["end_time"]), axis=1)
df_bikes["precip"] = df_bikes.apply(lambda x: get_precip(x["start_time"],x["end_time"]), axis=1)

In [12]:
df_bikes.tail(5)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,ride_duration_minutes,temperature,precip
788900,2017-12-31 22:56:00,2017-12-31 22:59:00,3045,3052,11870,Indego30,13th & Locust,9th & Locust,0 days 00:03:00,-10.0,0.0
788902,2017-12-31 23:05:00,2017-12-31 23:33:00,3070,3124,3708,Indego30,"Spring Garden Station, MFL",Race Street Pier,0 days 00:28:00,-10.0,0.0
788904,2017-12-31 23:18:00,2017-12-31 23:25:00,3033,3046,11933,Indego30,10th & Chestnut,2nd & Market,0 days 00:07:00,-10.0,0.0
788905,2017-12-31 23:39:00,2017-12-31 23:40:00,3163,3163,6725,Indego30,25th & Locust,25th & Locust,0 days 00:01:00,-10.0,0.0
788906,2017-12-31 23:41:00,2017-12-31 23:55:00,3163,3057,11864,Indego30,25th & Locust,Philadelphia Museum of Art,0 days 00:14:00,-10.0,0.0


In [13]:
df_2017weather_unique.tail(5)

,date_time,max_temp,min_temp,precip
Order,,,,
8755,2017-12-31 19:00:00,-8.9,-8.9,0.0
8756,2017-12-31 20:00:00,-8.9,-8.9,0.0
8757,2017-12-31 21:00:00,-9.4,-9.4,0.0
8758,2017-12-31 22:00:00,-10.0,-10.0,0.0
8759,2017-12-31 23:00:00,-10.0,-10.0,0.0


# Regressions

Now we are doing regressions

1. linear regression: amount of loan bikes for one entire day and if the day is weekday or not

For this we are creating a new dataframe to get the daily total amount of loan bikes

In [14]:
from datetime import timedelta

# array containing all dates of the year
start_2017 = datetime(2017, 1, 1)
add_day = timedelta(days = 1)

days_2017 = []

for i in range(0, 365):
    days_2017.append(start_2017)
    start_2017 += add_day

print(days_2017)

days = range(0, 365)
days_Series = pd.Series(days_2017, index=days)
days_Series

[datetime.datetime(2017, 1, 1, 0, 0), datetime.datetime(2017, 1, 2, 0, 0), datetime.datetime(2017, 1, 3, 0, 0), datetime.datetime(2017, 1, 4, 0, 0), datetime.datetime(2017, 1, 5, 0, 0), datetime.datetime(2017, 1, 6, 0, 0), datetime.datetime(2017, 1, 7, 0, 0), datetime.datetime(2017, 1, 8, 0, 0), datetime.datetime(2017, 1, 9, 0, 0), datetime.datetime(2017, 1, 10, 0, 0), datetime.datetime(2017, 1, 11, 0, 0), datetime.datetime(2017, 1, 12, 0, 0), datetime.datetime(2017, 1, 13, 0, 0), datetime.datetime(2017, 1, 14, 0, 0), datetime.datetime(2017, 1, 15, 0, 0), datetime.datetime(2017, 1, 16, 0, 0), datetime.datetime(2017, 1, 17, 0, 0), datetime.datetime(2017, 1, 18, 0, 0), datetime.datetime(2017, 1, 19, 0, 0), datetime.datetime(2017, 1, 20, 0, 0), datetime.datetime(2017, 1, 21, 0, 0), datetime.datetime(2017, 1, 22, 0, 0), datetime.datetime(2017, 1, 23, 0, 0), datetime.datetime(2017, 1, 24, 0, 0), datetime.datetime(2017, 1, 25, 0, 0), datetime.datetime(2017, 1, 26, 0, 0), datetime.datetime(20

0     2017-01-01
1     2017-01-02
2     2017-01-03
3     2017-01-04
4     2017-01-05
         ...    
360   2017-12-27
361   2017-12-28
362   2017-12-29
363   2017-12-30
364   2017-12-31
Length: 365, dtype: datetime64[ns]

In [15]:
#checks whether the weekday is a day in the week(1) or in weekend(0)
def is_weekday(day):
    if day == 5 or day == 6:
        return 0
    else:
        return 1


#sums up all loans for one entire day in 2017
def loans_per_day(month, day):
    day_in_2017_start = datetime(2017, month, day, 0, 0)
    day_in_2017_end = datetime(2017, month, day, 23, 59)
    df_day_start = df_bikes[df_bikes["start_time"] >= day_in_2017_start]
    df_day_end = df_day_start[df_day_start["start_time"] <= day_in_2017_end]
    return len(df_day_end)





lin_reg_bikes_df = pd.DataFrame({'date': pd.concat([days_Series])},
                                index=range(0, 365),
                                columns=['date'])

lin_reg_bikes_df['weekday'] = lin_reg_bikes_df['date'].apply(lambda x: x.weekday())
lin_reg_bikes_df['is_weekday'] = lin_reg_bikes_df['weekday'].apply(lambda x: is_weekday(x))
lin_reg_bikes_df['loans'] = lin_reg_bikes_df['date'].apply(lambda x: loans_per_day(x.month, x.day))


lin_reg_bikes_df.tail()

,date,weekday,is_weekday,loans
360,2017-12-27,2,1,637
361,2017-12-28,3,1,566
362,2017-12-29,4,1,566
363,2017-12-30,5,0,204
364,2017-12-31,6,0,241


In [17]:
rec=smf.ols(formula='loans~is_weekday', data=lin_reg_bikes_df).fit()
print(rec.summary())

                            OLS Regression Results                            
Dep. Variable:                  loans   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     16.40
Date:                Sun, 18 Jul 2021   Prob (F-statistic):           6.28e-05
Time:                        11:05:42   Log-Likelihood:                -3079.9
No. Observations:                 365   AIC:                             6164.
Df Residuals:                     363   BIC:                             6172.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1785.3143    109.386     16.321      0.0

In [18]:
from sklearn.linear_model import LinearRegression

linear_model_bikes = LinearRegression(fit_intercept=True, normalize=False)

linear_model_bikes.fit(lin_reg_bikes_df['is_weekday'].values.reshape((-1,1)), lin_reg_bikes_df['loans'])
print(linear_model_bikes.coef_, linear_model_bikes.intercept_)

[524.83186813] 1785.314285714286


In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#MSE and RMSE
print("Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['is_weekday'].values.reshape((-1,1)), lin_reg_bikes_df['loans']),"(Bikes)^2")
print("Root Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['is_weekday'].values.reshape((-1,1)), lin_reg_bikes_df['loans'])**0.5,"Bikes")

Mean Squared Error: 5964624.767123288 (Bikes)^2
Root Mean Squared Error: 2442.2581286840436 Bikes


2. linear regression: amount of bike loans and precipitation

In [ ]:
#sums up all rain hours for one entire day in 2017
def rain_hours(month, day):
    day_in_2017_start = datetime(2017, month, day, 0, 0)
    day_in_2017_end = datetime(2017, month, day, 23, 59)
    df_rain_day_start = df_2017weather_unique[df_2017weather_unique['date_time'] >= day_in_2017_start]
    df_rain_day_end = df_rain_day_start[df_rain_day_start['date_time'] <= day_in_2017_end]

    rain =  sum(1 for i in df_rain_day_end['precip'] if i == 1)

    return rain

#test
rain_hours(3, 7)

In [37]:
lin_reg_bikes_df['precip_hours'] = lin_reg_bikes_df['date'].apply(lambda x: rain_hours(x.month, x.day))
lin_reg_bikes_df.head()

,date,weekday,is_weekday,loans,mean_temp,precip_hours
0,2017-01-01,6,0,856,7.438785,0
1,2017-01-02,0,1,1510,4.177185,12
2,2017-01-03,1,1,1854,6.475405,9
3,2017-01-04,2,1,0,0.000000,2
4,2017-01-05,3,1,0,0.000000,0


In [38]:
rec2=smf.ols(formula='loans~precip_hours', data=lin_reg_bikes_df).fit()
print(rec2.summary())

                            OLS Regression Results                            
Dep. Variable:                  loans   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     37.16
Date:                Sun, 18 Jul 2021   Prob (F-statistic):           2.79e-09
Time:                        15:02:11   Log-Likelihood:                -3070.2
No. Observations:                 365   AIC:                             6144.
Df Residuals:                     363   BIC:                             6152.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2355.1128     65.550     35.928   

In [39]:
print("Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['precip_hours'].values.reshape((-1,1)), lin_reg_bikes_df['loans']),"(Bikes)^2")
print("Root Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['precip_hours'].values.reshape((-1,1)), lin_reg_bikes_df['loans'])**0.5,"Bikes")

Mean Squared Error: 5962106.42739726 (Bikes)^2
Root Mean Squared Error: 2441.7424981756903 Bikes


3. linear regression: duration of the biketrip and temperature

In [44]:
df_bikes['ride_duration_to_int_seconds'] = df_bikes['ride_duration_minutes'].apply(lambda x: (x.total_seconds()))
df_bikes.head()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,ride_duration_minutes,temperature,precip,ride_duration_to_int_seconds
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL",0 days 00:11:00,6.1,0.0,660.0
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:36:00,6.1,0.0,2160.0
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:35:00,6.1,0.0,2100.0
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS",0 days 00:12:00,6.1,0.0,720.0
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier,0 days 00:08:00,6.1,0.0,480.0


In [21]:
#eher nicht so machen, sondern wie in der folgenden Zelle
linear_model_duration = LinearRegression(fit_intercept=True, normalize=False)

linear_model_duration.fit(df_bikes['temperature'].values.reshape((-1,1)), df_bikes['ride_duration_minutes']/3600)
print(linear_model_duration.coef_, linear_model_duration.intercept_)

[2297914.24325896] 293491010.75289893


In [46]:
rec3=smf.ols(formula='ride_duration_to_int_seconds~temperature', data=df_bikes).fit()
print(rec3.summary())

                                 OLS Regression Results                                 
Dep. Variable:     ride_duration_to_int_seconds   R-squared:                       0.001
Model:                                      OLS   Adj. R-squared:                  0.001
Method:                           Least Squares   F-statistic:                     408.8
Date:                          Sun, 18 Jul 2021   Prob (F-statistic):           7.23e-91
Time:                                  17:56:36   Log-Likelihood:            -7.4448e+06
No. Observations:                        788096   AIC:                         1.489e+07
Df Residuals:                            788094   BIC:                         1.489e+07
Df Model:                                     1                                         
Covariance Type:                      nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

4. linear regression: amount of all loans for a day and mean temperature of all loans on that day

In [23]:
#sums up all loans for one entire day in 2017
def mean_temperature_loans(month, day):
    day_in_2017_start = datetime(2017, month, day, 0, 0)
    day_in_2017_end = datetime(2017, month, day, 23, 59)
    df_day_start = df_bikes[df_bikes["start_time"] >= day_in_2017_start]
    df_day_end = df_day_start[df_day_start["start_time"] <= day_in_2017_end]

    temp = 0
    for index, row in df_day_end.iterrows():
        temp += row['temperature']

    if len(df_day_end) > 0:
        mean_temp = temp / len(df_day_end)
    else:
        mean_temp = 0

    return mean_temp

mean_temperature_loans(1, 3)

6.475404530744328

In [24]:
lin_reg_bikes_df['mean_temp'] = lin_reg_bikes_df['date'].apply(lambda x: mean_temperature_loans(x.month, x.day))

In [25]:
lin_reg_bikes_df.head()

,date,weekday,is_weekday,loans,mean_temp
0,2017-01-01,6,0,856,7.438785
1,2017-01-02,0,1,1510,4.177185
2,2017-01-03,1,1,1854,6.475405
3,2017-01-04,2,1,0,0.000000
4,2017-01-05,3,1,0,0.000000


In [26]:
rec4 = smf.ols(formula='loans~mean_temp', data=lin_reg_bikes_df).fit()
print(rec4.summary())

                            OLS Regression Results                            
Dep. Variable:                  loans   R-squared:                       0.637
Model:                            OLS   Adj. R-squared:                  0.636
Method:                 Least Squares   F-statistic:                     636.3
Date:                Sun, 18 Jul 2021   Prob (F-statistic):           7.89e-82
Time:                        11:26:54   Log-Likelihood:                -2903.1
No. Observations:                 365   AIC:                             5810.
Df Residuals:                     363   BIC:                             5818.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    796.5728     64.998     12.255      0.0

In [27]:
print("Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['mean_temp'].values.reshape((-1,1)), lin_reg_bikes_df['loans']),"(Bikes)^2")
print("Root Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['mean_temp'].values.reshape((-1,1)), lin_reg_bikes_df['loans'])**0.5,"Bikes")

Mean Squared Error: 5888231.952722154 (Bikes)^2
Root Mean Squared Error: 2426.56793696821 Bikes


5. linear regression: amount of all loans for a day and weekday

In [28]:
rec5=smf.ols(formula='loans~weekday', data=lin_reg_bikes_df).fit()
print(rec5.summary())

                            OLS Regression Results                            
Dep. Variable:                  loans   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     7.759
Date:                Sun, 18 Jul 2021   Prob (F-statistic):            0.00562
Time:                        11:26:54   Log-Likelihood:                -3084.1
No. Observations:                 365   AIC:                             6172.
Df Residuals:                     363   BIC:                             6180.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2407.3987    107.069     22.485      0.0

In [29]:
print("Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['weekday'].values.reshape((-1,1)), lin_reg_bikes_df['loans']),"(Bikes)^2")
print("Root Mean Squared Error:",mean_squared_error(lin_reg_bikes_df['weekday'].values.reshape((-1,1)), lin_reg_bikes_df['loans'])**0.5,"Bikes")

Mean Squared Error: 5955600.180821918 (Bikes)^2
Root Mean Squared Error: 2440.4098386996225 Bikes
